# How to use tsma with Inputs Outputs model?

All the example are done with threesector model

## 1. The model format and savings

### model inputs

The model format is create base on the Model class from tsma.models.model

A model take as inputs three dictionaries : 
 - parameters, hyper_parameters, initial_values

In [1]:
import time as t
import numpy as np
from models.threesector import ThreeSectorModel as model

parameters= {
        # Sectoral parameters
        "lprod": [3.0, 3.0, 3.0],
        "kprod": [1.0, 1.0, 1.0],
        "tgtupdate": [1, 1, 1],
        "cutgt": [0.8, 0.8, 0.8],
        "inv_cov": [1.0, 1.0, 1.0],
        "markup": [1.0, 1.0, 1.0],
        "deprate_k": np.array([0.1, 0.1, 0.1]),
        "deprate_inv": [0, 1, 1],
        # Leontief Matrix
        "zmat": np.array(
            [
                [0.1, 0.1, 0.1],  # E for E, R for E, G for E
                [0.1, 0.1, 0.1],
                [0.1, 0.1, 0.1],
            ]  # E for G, G for G
        ),
        # Households
        "mincons": [0, 0, 0],
        "elast_income": np.array([0.0, 0.0, 0.0]),  # 0.1,
        "elast_price": np.array(
            [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]
        ),
        # Other parameters
        "interest": 0.0,
        "participation_nat": 0.95,
        "wage_part": 0.0,
        "wage_infl": 0.1,
        "pop_grow": 0.0,
    }
initial_values={
    # Float variables
    "resources": 3e3,
    "population": 1e3,
    "wage": 1,
    "income_hh": 10,
    "deposits_hh": 0,
    "cpi": 0,
    # Vector avriables
    "prod_tgt": np.array([0, 0, 0]),
    "prod": np.array([0, 0, 0]),
    "dem_labor": np.array([1, 1, 1]),
    "labor": np.array([1, 1, 1]),
    "capital": np.array([10, 10, 10]),
    "debt": np.array([0, 0, 0]),
    "prices": np.array([1, 1, 1]),
    "income": np.array([0, 0, 0]),
    "deposits": np.array([0, 0, 0]),
    "dem_hh": np.array([10, 10, 10]),
    "dem_hh_tgt": np.array([10, 10, 10]),
    "cons_hh": np.array([0.1, 0.1, 0.1]),
    # Matrix variables
    "dem_interm": np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
    "dem_invest": np.zeros((3, 3)),
    "cons_interm": np.zeros((3, 3)),
    "investment": np.zeros((3, 3)),
    "inventory": np.array([[0, 0, 0], [0, 2, 0], [0, 0, 2]]),
}
hyper_parameters={
    "sectors": ["en", "re", "go"],
    "n_sectors": 3,
    "t_end": 103,
    "seed": 0,
}

m = model(parameters, hyper_parameters, initial_values)

### model outputs

Then a method **simulate** gives the outputs of the simulations with a Dataframe.
For more information see Gross2022 code to build your own model with the right format.


In [2]:
output = m.simulate(t_end = 40, seed = 10)
varnames = list(output)
nvar = len(varnames)
output.tail()

,resources,population,wage,income_hh,deposits_hh,cpi,prod_tgt_en,prod_tgt_re,prod_tgt_go,expected_sales_en,...,investment_go_go,inventory_en_en,inventory_en_re,inventory_en_go,inventory_re_en,inventory_re_re,inventory_re_go,inventory_go_en,inventory_go_re,inventory_go_go
35,2981.276777,1000.0,1.244514,0.346135,4.008600,-0.020661,0.335956,0.331390,37202.281516,0.335956,...,-6.938844e-18,0.0,0.000000e+00,0.003266,0.0,0.049702,0.007237,0.0,0.004245,0.049840
36,2981.241694,1000.0,1.248688,0.312566,3.658327,0.033545,0.307196,0.302951,46503.231823,0.307196,...,3.469428e-18,0.0,0.000000e+00,0.002826,0.0,0.041357,0.006605,0.0,0.004023,0.041466
37,2981.209466,1000.0,1.257443,0.283282,3.371999,0.070112,0.272573,0.268549,58129.389751,0.272573,...,-3.469434e-18,0.0,-4.336809e-19,0.002471,0.0,0.035429,0.006001,0.0,0.003750,0.035519
38,2981.178392,1000.0,1.264730,0.256431,3.106707,0.057947,0.237424,0.233674,72662.034277,0.237424,...,-3.469438e-18,0.0,-4.336809e-19,0.002167,0.0,0.032421,0.005342,0.0,0.003375,0.032500
39,0.000000,1000.0,0.000000,0.000000,2.832646,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.0,4.336809e-19,0.001890,0.0,0.031172,0.004649,0.0,0.002939,0.031246


### parameters format


Floats and all types of numerical arrays are accepted. 
However, the hyperparameters have to be integers or integer arrays.
For input output and ABM models, agent indexes can be encoded into the databases:
- **names_to_index** create an index system which is optional 
- **encode** and **decode** detect if an agent index is used instead of a simple coordinate based on the format.

In [3]:
from basics.text_management import names_to_index

# agent ids indexes
agent_ids = names_to_index(hyper_parameters["sectors"])
print(f"agent ids {agent_ids} \n")


agent ids ['E' 'R' 'G'] 



In [4]:
from basics.text_management import encode, decode

print("encoded with agent ids---------------\n")
for code in list(encode(parameters, hyper_parameters, initial_values, agent_ids)):
    print(f"{code} \n")

encoded with agent ids---------------

p1__lprod__E 

p1__lprod__R 

p1__lprod__G 

p1__kprod__E 

p1__kprod__R 

p1__kprod__G 

p1__tgtupdate__E 

p1__tgtupdate__R 

p1__tgtupdate__G 

p1__cutgt__E 

p1__cutgt__R 

p1__cutgt__G 

p1__inv_cov__E 

p1__inv_cov__R 

p1__inv_cov__G 

p1__markup__E 

p1__markup__R 

p1__markup__G 

p1__deprate_k__E 

p1__deprate_k__R 

p1__deprate_k__G 

p1__deprate_inv__E 

p1__deprate_inv__R 

p1__deprate_inv__G 

p1__zmat__E_E 

p1__zmat__E_R 

p1__zmat__E_G 

p1__zmat__R_E 

p1__zmat__R_R 

p1__zmat__R_G 

p1__zmat__G_E 

p1__zmat__G_R 

p1__zmat__G_G 

p1__mincons__E 

p1__mincons__R 

p1__mincons__G 

p1__elast_income__E 

p1__elast_income__R 

p1__elast_income__G 

p1__elast_price__E_E 

p1__elast_price__E_R 

p1__elast_price__E_G 

p1__elast_price__R_E 

p1__elast_price__R_R 

p1__elast_price__R_G 

p1__elast_price__G_E 

p1__elast_price__G_R 

p1__elast_price__G_G 

p1__interest 

p1__participation_nat 

p1__wage_part 

p1__wage_infl 

p1__pop_gro

### automatic output management
Here the outputs are saved on SQL databases. The overwrite option can be used to replace the existing data associated to the parameters.

In [5]:
import numpy as np
output = m.simulate(t_end = 40, seed = 10)

print(np.shape(output))
print(output.iloc[:3, :6])

  0%|          | 0/38 [00:00<?, ?it/s]C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:435: RuntimeWarning: divide by zero encountered in true_divide
  pdem = np.nan_to_num(np.minimum(1, avail / tot_dem))
C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:435: RuntimeWarning: invalid value encountered in true_divide
  pdem = np.nan_to_num(np.minimum(1, avail / tot_dem))
100%|██████████| 38/38 [00:00<00:00, 907.20it/s]

(40, 96)
     resources  population      wage  income_hh  deposits_hh       cpi
0  3000.000000      1000.0  1.000000  10.000000     0.000000  0.000000
1  2997.085020      1000.0  1.000000   8.666667     0.000000  0.000000
2  2994.552897      1000.0  0.994204   6.620663     4.450929 -0.057959


## 2 First visualizations of the outputs

Simple plots of the time series

In [4]:
from visuals.figures import mosaique

fig = mosaique(output, 6, "test 1 sim", width = 1600, height = 3000)
fig.show()

In [9]:
from basics.transfers import auto_dct_groups

dct_groups = auto_dct_groups(varnames, hyper_parameters["sectors"])
dct_groups

{'dem interm en': ['dem_interm_en_en', 'dem_interm_en_re', 'dem_interm_en_go'],
 'dem interm re': ['dem_interm_re_en', 'dem_interm_re_re', 'dem_interm_re_go'],
 'dem interm go': ['dem_interm_go_en', 'dem_interm_go_re', 'dem_interm_go_go'],
 'dem invest en': ['dem_invest_en_en', 'dem_invest_en_re', 'dem_invest_en_go'],
 'dem invest re': ['dem_invest_re_en', 'dem_invest_re_re', 'dem_invest_re_go'],
 'dem invest go': ['dem_invest_go_en', 'dem_invest_go_re', 'dem_invest_go_go'],
 'cons interm en': ['cons_interm_en_en',
  'cons_interm_en_re',
  'cons_interm_en_go'],
 'cons interm re': ['cons_interm_re_en',
  'cons_interm_re_re',
  'cons_interm_re_go'],
 'cons interm go': ['cons_interm_go_en',
  'cons_interm_go_re',
  'cons_interm_go_go'],
 'investment en': ['investment_en_en', 'investment_en_re', 'investment_en_go'],
 'investment re': ['investment_re_en', 'investment_re_re', 'investment_re_go'],
 'investment go': ['investment_go_en', 'investment_go_re', 'investment_go_go'],
 'inventory en':

## 3 figures saving

To **save** plotly figures use :
- **get_save_path**(model, prev_folder : bool = False) -> str
- **save_fig** (fig, newpath_fig: str, figname: str, save_format: str = "html")

In [6]:
import os
from collect.output_management import  get_save_path 
from visuals.fig_management import save_fig
from visuals.figures import mosaique

fig = mosaique(output, 6, "test 1 sim", width = 1600, height = 3000)
path = os.sep.join([get_save_path(m, False), "figures"])

save_fig(fig, newpath_fig = path, figname = "test_save", save_format = "png")
print(path)

C:\Users\samud\Bureau\Python code\Repo\tsma\Outputs\threesectors\figures


## 4 An adaptive dashboard for manual parameters explorations 


Controles :
- all parameters : parameters, hyper_parameters, initial_values
- parameters_Sliders [ min, max, number of steps ] If empty there is no sliders mode
        
To create  **parameters_Sliders** use **encode** from basics.text_management to automatically have the list of the encoded variable names.

A variable can be used as a reference with **var_compared**. Here unemployement is used by default.

In [7]:

params_sliders = {
        "p1__lprod__E": [0, 4, 20],
        "p1__lprod__R": [0, 4, 20],
        "p1__lprod__G": [0, 4, 20],
        "p1__kprod__E": [0, 40, 200],
        "p1__kprod__R": [0, 40, 200],
        "p1__kprod__G": [0, 40, 200],
        "p1__tgtupdate__E": [0, 2, 20],
        "p1__tgtupdate__R": [0, 2, 20],
        "p1__tgtupdate__G": [0, 2, 20],
        "p1__cutgt__E": [0, 1, 20],
        "p1__cutgt__R": [0, 1, 20],
        "p1__cutgt__G": [0, 1, 20],
        "p1__inv_cov__E": [0, 2, 20],
        "p1__inv_cov__R": [0, 2, 20],
        "p1__inv_cov__G": [0, 2, 20],
        "p1__markup__E": [1, 2, 20],
        "p1__markup__R": [1, 2, 20],
        "p1__markup__G": [1, 2, 20],
        "p1__deprate_k__E": [0, 1, 20],
        "p1__deprate_k__R": [0, 1, 20],
        "p1__deprate_k__G": [0, 1, 20],
        "p1__deprate_inv__E": [0, 1, 20],
        "p1__deprate_inv__R": [0, 1, 20],
        "p1__deprate_inv__G": [0, 1, 20],
        "p1__zmat__E_E": [0, 1, 100],
        "p1__zmat__E_R": [0, 10, 100],
        "p1__zmat__E_G": [0, 10, 100],
        "p1__zmat__R_E": [0, 10, 100],
        "p1__zmat__R_R": [0, 1, 100],
        "p1__zmat__R_G": [0, 10, 100],
        "p1__zmat__G_E": [0, 10, 100],
        "p1__zmat__G_R": [0, 10, 100],
        "p1__zmat__G_G": [0, 1, 100],
        "p1__mincons__E": [0, 10, 2000],
        "p1__mincons__R": [0, 10, 2000],
        "p1__mincons__G": [0, 10, 2000],
        "p1__elast_income__E": [0, 2, 20],
        "p1__elast_income__R": [0, 2, 20],
        "p1__elast_income__G": [0, 2, 20],
        "p1__elast_price__E_E": [0, 2, 20],
        "p1__elast_price__E_R": [0, 2, 20],
        "p1__elast_price__E_G": [0, 2, 20],
        "p1__elast_price__R_E": [0, 2, 20],
        "p1__elast_price__R_R": [0, 2, 20],
        "p1__elast_price__R_G": [0, 2, 20],
        "p1__elast_price__G_E": [0, 2, 20],
        "p1__elast_price__G_R": [0, 2, 20],
        "p1__elast_price__G_G": [0, 2, 20],
        "p1__interest": [0, 1, 20],
        "p1__participation_nat": [0, 1, 20],
        "p1__wage_part": [0, 10, 100],
        "p1__wage_infl": [0, 10, 100],
        "p1__pop_grow": [0, 1, 100],

    
        "p2__n_sectors": [0, 10, 10],
        "p2__t_end": [0, 1000, 1000],
        "p2__seed": [0, 1000, 1000],
        
        "p3__resources": [0, 100000000000.0, 1000000.0],
        "p3__population": [0, 10000, 1000],
        "p3__wage": [0, 100, 1000],
        "p3__income_hh": [0, 1000, 1000],
        "p3__deposits_hh": [0, 10000, 1000],
        "p3__cpi": [0, 100, 1000],
        "p3__prod_tgt__E": [0, 100, 1000],
        "p3__prod_tgt__R": [0, 100, 1000],
        "p3__prod_tgt__G": [0, 100, 1000],
        "p3__prod__E": [0, 100, 1000],
        "p3__prod__R": [0, 100, 1000],
        "p3__prod__G": [0, 100, 1000],
        "p3__dem_labor__E": [0, 1000, 1000],
        "p3__dem_labor__R": [0, 1000, 1000],
        "p3__dem_labor__G": [0, 1000, 1000],
        "p3__labor__E": [0, 100, 1000],
        "p3__labor__R": [0, 100, 1000],
        "p3__labor__G": [0, 100, 1000],
        "p3__capital__E": [0, 500000, 1000],
        "p3__capital__R": [0, 500000, 1000],
        "p3__capital__G": [0, 500000, 1000],
        "p3__debt__E": [0, 500000, 1000],
        "p3__debt__R": [0, 500000, 1000],
        "p3__debt__G": [0, 500000, 1000],
        "p3__prices__E": [0, 100, 1000],
        "p3__prices__R": [0, 100, 1000],
        "p3__prices__G": [0, 100, 1000],
        "p3__income__E": [0, 1000, 1000],
        "p3__income__R": [0, 1000, 1000],
        "p3__income__G": [0, 1000, 1000],
        "p3__deposits__E": [0, 500000, 1000],
        "p3__deposits__R": [0, 500000, 1000],
        "p3__deposits__G": [0, 500000, 1000],
        "p3__dem_hh__E": [0, 100, 1000],
        "p3__dem_hh__R": [0, 100, 1000],
        "p3__dem_hh__G": [0, 100, 1000],
        "p3__dem_hh_tgt__E": [0, 100, 1000],
        "p3__dem_hh_tgt__R": [0, 100, 1000],
        "p3__dem_hh_tgt__G": [0, 100, 1000],
        "p3__cons_hh__E": [0, 100, 1000],
        "p3__cons_hh__R": [0, 100, 1000],
        "p3__cons_hh__G": [0, 100, 1000],
        "p3__dem_interm__E_E": [0, 100, 1000],
        "p3__dem_interm__E_R": [0, 100, 1000],
        "p3__dem_interm__E_G": [0, 100, 1000],
        "p3__dem_interm__R_E": [0, 100, 1000],
        "p3__dem_interm__R_R": [0, 100, 1000],
        "p3__dem_interm__R_G": [0, 100, 1000],
        "p3__dem_interm__G_E": [0, 100, 1000],
        "p3__dem_interm__G_R": [0, 100, 1000],
        "p3__dem_interm__G_G": [0, 100, 1000],
        "p3__dem_invest__E_E": [0, 100, 1000],
        "p3__dem_invest__E_R": [0, 100, 1000],
        "p3__dem_invest__E_G": [0, 100, 1000],
        "p3__dem_invest__R_E": [0, 100, 1000],
        "p3__dem_invest__R_R": [0, 100, 1000],
        "p3__dem_invest__R_G": [0, 100, 1000],
        "p3__dem_invest__G_E": [0, 100, 1000],
        "p3__dem_invest__G_R": [0, 100, 1000],
        "p3__dem_invest__G_G": [0, 100, 1000],
        "p3__cons_interm__E_E": [0, 100, 1000],
        "p3__cons_interm__E_R": [0, 100, 1000],
        "p3__cons_interm__E_G": [0, 100, 1000],
        "p3__cons_interm__R_E": [0, 100, 1000],
        "p3__cons_interm__R_R": [0, 100, 1000],
        "p3__cons_interm__R_G": [0, 100, 1000],
        "p3__cons_interm__G_E": [0, 100, 1000],
        "p3__cons_interm__G_R": [0, 100, 1000],
        "p3__cons_interm__G_G": [0, 100, 1000],
        "p3__investment__E_E": [0, 100, 1000],
        "p3__investment__E_R": [0, 100, 1000],
        "p3__investment__E_G": [0, 100, 1000],
        "p3__investment__R_E": [0, 100, 1000],
        "p3__investment__R_R": [0, 100, 1000],
        "p3__investment__R_G": [0, 100, 1000],
        "p3__investment__G_E": [0, 100, 1000],
        "p3__investment__G_R": [0, 100, 1000],
        "p3__investment__G_G": [0, 100, 1000],
    
        "p3__inventory__E_E": [0, 100, 1000],
        "p3__inventory__E_R": [0, 100, 1000],
        "p3__inventory__E_G": [0, 100, 1000],
        "p3__inventory__R_E": [0, 100, 1000],
        "p3__inventory__R_R": [0, 100, 1000],
        "p3__inventory__R_G": [0, 100, 1000],
        "p3__inventory__G_E": [0, 100, 1000],
        "p3__inventory__G_R": [0, 100, 1000],
        "p3__inventory__G_G": [0, 100, 1000],
    }


### Change the Latex
with **f_latex** for the latex traduction of the parameters

and **ncol_latex** for the number of columns of parameters on the sidebar

### Change the figure
with **f_fig** for the figure plotted.


In [8]:
from basics.text_management import varname_to_latex
from visuals.figures import name_to_rgb, adapted_mosaique, f_line_em

def threesec_f_latex(varname):
    core_conv = {
        "popgrow": "$ g $",
        "zmat": "Z",
        "cons": "\mathcal{C}",
        "dem invest": "\mathcal{I}",
        "investment": "\mathcal{I}",
        "capital": "\mathcal{K}",
        "dem": "\mathcal{D}",
        "prod": "\mathcal{X}",
        "income": "Y",
        "labor": "\mathcal{L}",
        "dem labor": "\mathcal{L}",
        "inventory": "\mathcal{Q}",
        "resources": "\mathcal{R}",
        "deposits": "M",
        "prices": "P",
        "debt": "D",
        "wage": "w",
        "interest": "r",
        "lprod": "\ alpha",
        "kprod": "\ kappa",
        "tgtupdate": "\ gamma",
        "cutgt": "CU",
        "markup": "\ mu",
        "deprate k": "\ delta",
        "deprate inv": "\ delta",
        "elast price": "\ phi",
        "elast income": "\ phi",
        "inv cov": "\ eta",
        "mincons": "\ rho",
        "participation nat": "\lambda",
        "wage part": "\ omega",
        "wage infl": "\ omega",
        "population": "\mathcal{N}",
        "pop grow": "\mathit{n}",
        "cpi": "CPI",
        "t end": "\mathcal{T}",
    }
    exponents = ["hh", "tgt", "interm"]
    indexes = ['en', 're', 'go', 'E', 'R', 'G'] 

    special_exp = {
        "dem invest": "dem",
        "dem labor": "dem",
        "cutgt": "tgt",
    }
    special_index = {
        "deprate k": "\mathcal{K}",
        "deprate inv": "\mathcal{Q}",
        "elast price": "p",
        "elast income": "Y",
        "inv cov": "\mathcal{Q}",
        "mincons": "0",
        "participation nat": "0",
        "wage part": "\lambda",
        "wage infl": "\pi",
    }
    return varname_to_latex(
        varname, exponents, indexes, core_conv, special_exp, special_index
    )


def threesec_f_fig(output, title, dct_groups, nvar):
    return adapted_mosaique(
    output,
    title,
    dct_groups,
    f_line = f_line_em,
    yaxis_to_right = [],
    nskip = 1,
    width = 1600,
    vspace = 0.025,
    hspace = 0.05
    )


In [13]:
fig = threesec_f_fig(output, "test", dct_groups, 1)
fig.show()

In [9]:
from visuals.dashboards import app_mosaique, run_app

if __name__ == "__main__":
    app = app_mosaique(
        parameters,
        hyper_parameters,
        initial_values,
        params_sliders,
        model,
        dct_groups=dct_groups,
        in_jupyter=True,
        ncol_latex = 3,

        f_latex=threesec_f_latex,
        f_fig=threesec_f_fig,
    )
    run_app(app, in_jupyter = True)

Dash app running on http://127.0.0.1:8050/


### Add an image 
with **app_mos_and_image**, the image is computed at the first update of the dashboard.

### Change the image

with **f_img**. The default function computed Okun's and Phillips's curves. 
WARNING : the duration of the simulation has to be sufficient to caculate these curves, and clear the cache or change f_img name to update the img.


In [11]:
from visuals.figures import my_heatmap
from visuals.fig_management import save_fig
from visuals.dashboards import app_mos_and_image, run_app

    
def heatmap_f_img(m, save_path, img_format):
    
    df = m.output.iloc[3:, :]
    fig = my_heatmap(df, "new_img")
    fig.savefig(save_path, dpi=200, format=img_format)


if __name__ == "__main__":

    app = app_mos_and_image(
        parameters,
        hyper_parameters,
        initial_values,
        params_sliders,
        model,
        dct_groups,
        in_jupyter=True,
        f_img = heatmap_f_img,
        f_latex=threesec_f_latex,
        f_fig=threesec_f_fig,
    )
    run_app(app, in_jupyter = True)

Dash app running on http://127.0.0.1:8050/
inputs
inputs
inputs


## 5 multi-simulations computation and queries

###  multiple simulations

 **n_sim_multiproc** to parallelize the computations of simulation. 
 However this function need to know the number of variable generated by the model.

In [7]:
from collect.iterators import nsim_mproc
if __name__ == "__main__":

    t_end = 40
    t1 = t.time()
    outputs = nsim_mproc(m, nvar, t_end, nsim = 5, save = True, overwrite = True)
    t2 = t.time()
    print(t2 - t1)
    

number of cores used 5
number of pools 1


100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


shape of the pool outputs: (5, 40, 96)
targeted shape: (5, 40, 96)
1.3852920532226562


### databases queries from a list of ids

In [6]:
from collect.output_management import  query_parameters_specific, query_simulations
t1 = t.time()
df_params = query_parameters_specific(m, [0,3,5,6], t_end = 40)
t2 = t.time()
outputs = query_simulations(m, df_params["sim_id"])
t3 = t.time()
print(t2-t1, t3 - t2)
print()

100%|██████████| 1/1 [00:00<00:00, 142.94it/s]

0.030916690826416016 0.021950244903564453



## 6 multi-simulations plots

### Simple plots

In [9]:
from visuals.figures import mosaique_nsim, mosaique_hist_nsim

fig = mosaique_nsim(outputs, 6, "test_nsim_lines", varnames)
fig.show()

fig = mosaique_hist_nsim(outputs, 6, "test_nsim_hist", varnames)
fig.show()


### Transient Analysis

Confident Interval visualization of significance 99%

In [16]:
from visuals.figures import mosaique_transient, adapted_mosaique_transient

fig = mosaique_transient(outputs, 0.01, 6, "test_nsim_transient", varnames)
fig.show()

fig = adapted_mosaique_transient(outputs,"test_nsim_transient", varnames, dct_groups,  sign = 0.01, ncol = 6)
fig.show()


## 7 Data collection

### 1 set of parameters collection

In [23]:
from visuals.figures import adapted_mosaique_transient, f_line_em
from visuals.fig_management import save_fig

def collect_fig_3sec(
    parameters,
    hyper_parameters,
    initial_values,
    outputs,
    varnames,
    dct_groups,
    sim_id,
    path_figures,
    fig_format,
):

    title = "transient_analysis" + sim_id


    fig = adapted_mosaique_transient(
        outputs,
        title,
        varnames,
        dct_groups,
        sign=0.1,
        ncol=6,
        width=1700,
        height=2000,
        hspace=0.09,
        vspace=0.03,
        f_line=f_line_em,
        yaxis_to_right=[],
    )
    save_fig(fig, path_figures, "transient_analysis" + sim_id, fig_format)

In [11]:
from collect.data_collect import oneset_collect

if __name__ == "__main__":
    hyper_parameters["t_end"] = 40
    outputs2 =oneset_collect(
            model,
            parameters,
            initial_values,
            hyper_parameters,
            
            nsim=10,
            seeds=[],
            maxcore=-1,
            
            save=False,
            overwrite=False,
            save_fig=True,
            fig_format="png",
            nskip = 9,
            dct_groups=dct_groups,
        )


number of cores used 10
number of pools 1


100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


shape of the pool outputs: (10, 40, 96)
targeted shape: (10, 40, 96)


In [12]:
m.hyper_parameters = {
    "sectors": ["en", "re", "go"],
    "n_sectors": 3,
    "t_end": 103,
    "seed": 0,
}
m_ref = m

In [13]:
from collect.data_collect import para_exploration
para_ranges = {
    "p1__lprod__E": [0, 20],
    "p1__lprod__R": [0, 20],
    "p1__lprod__G": [0, 20],

    
    "p3__deposits__E": [0, 500000],
    "p3__deposits__R": [0, 500000],
    "p3__deposits__G": [0, 500000],
}


if __name__ == "__main__":

    para_exploration(
        model,
        m_ref,
        nvar,
        nsim=10,
        dct_bounds=para_ranges,
        dct_groups = dct_groups,
        nmin = 1,
        nskip = 9,
        
    )


  0%|          | 0/101 [00:00<?, ?it/s]C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:435: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:435: RuntimeWarning:

invalid value encountered in true_divide

100%|██████████| 101/101 [00:00<00:00, 1948.77it/s]


number of cores used 10
number of pools 1


100%|██████████| 1/1 [00:02<00:00,  2.50s/it]
C:\Users\samud\Bureau\Python code\Repo\tsma\collect\data_collect.py:113: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.




shape of the pool outputs: (10, 103, 96)
targeted shape: (10, 103, 96)
----------------------------------------------------------------------
number of sets of parameters 27
----------------------------------------------------------------------


  0%|          | 0/101 [00:00<?, ?it/s]C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:435: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:594: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:594: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:369: RuntimeWarning:

invalid value encountered in matmul

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:370: RuntimeWarning:

invalid value encountered in multiply

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:542: RuntimeWarning:

invalid value encountered in multiply

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.py:545: RuntimeWarning:

invalid value encountered in multiply

C:\Users\samud\Bureau\Python code\Repo\tsma\models\threesector.

number of cores used 10
number of pools 1



  0%|          | 0/27 [00:02<?, ?it/s]


IntegrityError: UNIQUE constraint failed: parameters_table.sim_id